In [22]:
# From Longhorn 4074
# Any version that accepts TCP8W-T8PQJ-WWRRH-QH76C-99FBW will work
key_data = {
    "p": 7181106593102322766813520532476531209871483588988471009176871145241389568314039093657656718839885029493125387894856821599452867350054864568294961595970889,
    "a": 1,
    "b": 0,
    "B": [
        520282615406607935808830413235837609227529008118239433194891765554084261177667142590192616462797266047427714603514505726507565809100858610756034340614180,
        4557046395510954851157569206449480560848332315791566919607580280750304632075435589109908909351625686398512699199297926705742962219032991805095344264722444
    ],
    "K": [
        1748427561645745685508888890965804844329037567281415535239953290167653001827496844268667372126127464466687812723744919132659150838866693283679107969476861,
        6808711632346399211426562555523956018872055718394662707289722207520029794097689415773036615424757895159410496488301598927496012713658489637493990459415502
    ],
    "order": 4633201844252750473,
    "private_key": 4329540238250287790
}

Product Key Generator, run above cell first

In [30]:
import hashlib

# p = order of field Fp
# Fp = Galois field of order p
# E = Elliptic curve y^2 = x^3 + ax + b over Fp
# B = generator on E
# K = inverse of public key
# order = order of E
# Ro = Ring Z/orderZ

p = key_data["p"]
Fp = GF(p)
E = EllipticCurve(Fp, [0, 0, 0, key_data["a"], key_data["b"]])
B = E.point(key_data["B"])
K = E.point(key_data["K"])
order = key_data["order"]
Ro = Integers(order)
private_key = -key_data["private_key"] % order

# OS Family of product key
# x64 VLK - 652
# x64 Retail - 306
os_family = 105

# Key alphabet
KCHARS = "BCDFGHJKMPQRTVWXY2346789"

def int_to_bytes(n, l=None):
    n = int(n)
    
    if not l:
        l = (n.bit_length() + 7) // 8
    
    return n.to_bytes(l, byteorder="little")

def encode_pkey(n):
    out = ""
    
    for i in range(25):
        out = KCHARS[n % 24] + out
        n //= 24
    
    out = "-".join([out[i:i+5] for i in range(0, len(out), 5)])
    return out

os_family <<= 1

while True:
    k = getrandbits(512)
    prefix = getrandbits(32) & 0x3ff
    
    r = k * B
    x, y = r.xy()

    mde = hashlib.sha1(b"\x79" + int_to_bytes(os_family, 2) + int_to_bytes(x, 64) + int_to_bytes(y, 64)).digest()
    e = int.from_bytes(mde[:4], byteorder="little")
    e &= 0x7fffffff
    
    mdh = hashlib.sha1(b"\x5d" + int_to_bytes(os_family, 2) + int_to_bytes(e, 4) + int_to_bytes(prefix, 4)).digest()
    h1 = int.from_bytes(mdh[:4], byteorder="little")
    h2 = int.from_bytes(mdh[4:8], byteorder="little") >> 2
    h2 &= 0x3fffffff
    h = h2 << 32 | h1
    b = Ro(-h * private_key)
    
    try:
        s = Ro(b)
        s = int((-b + sqrt(b^2 + 4 * Ro(k))) / 2)
    except:
        continue
    
    if s % 2 == 1:
        s += order
    
    if (s * (s * B + h * K)) != (s * (s * B + int(b) * B)):
        continue
    
    raw_pkey = prefix << 104 | s << 42 | e << 11 | os_family
    
    print((raw_pkey >> 11) & 0x7fffffff, e, (raw_pkey >> 42) & 0x3fffffffffffffff, s)
    
    # I could fix whatever bug made this necessary, but it works so I don't care
    if ((raw_pkey >> 11) & 0x7fffffff) != e or ((raw_pkey >> 42) & 0x3fffffffffffffff) != s:
        continue
    
    if (raw_pkey >> 32) & 0xffffffff < 0x40000000:
        break

print(hex(prefix)[2:], hex(s)[2:], hex(e)[2:], hex(os_family)[2:])
print(encode_pkey(raw_pkey))
pkey = encode_pkey(raw_pkey)

10928323 10928323 2003509697754295848 6615195716181683752
868186915 868186915 878552257861989116 5490238276289377020
446954708 446954708 776512975037445878 776512975037445878
353785506 353785506 1772257197849916146 6383943216277304050
1598465793 1598465793 385978941068063200 4997664959495451104
1118655069 1118655069 374356218385227934 374356218385227934
52 531faff484f609e 42ad525d d2
CH89R-TPQRK-GPJMW-7KTYQ-F8PJD


Product Key Verifier (must run above cell first)

In [29]:
def decode_pkey(k):
    k = k.replace("-", "")
    out = 0
    
    for c in k:
        out *= 24
        out += KCHARS.index(c)
    
    return out

pkey = input("Product Key (dashes optional): ")
print(pkey)
raw_key = decode_pkey(pkey)

osf = raw_key & 0x7ff
e = (raw_key >> 11) & 0x7fffffff
s = (raw_key >> 42) & 0x3fffffffffffffff
pf = (raw_key >> 104) & 0x3ff

mdh = hashlib.sha1(b"\x5d" + int_to_bytes(osf, 2) + int_to_bytes(e, 4) + int_to_bytes(pf, 4)).digest()
h1 = int.from_bytes(mdh[:4], byteorder="little")
h2 = int.from_bytes(mdh[4:8], byteorder="little") >> 2
h2 &= 0x3fffffff
h = h2 << 32 | h1

print(hex(pf)[2:], hex(s)[2:], hex(e)[2:], hex(osf)[2:])
print(hex(h)[2:])
print(bin(raw_key)[2:])
print(bin(pf << 104 | s << 42 | e << 11 | osf)[2:])

v = s * (s * B + h * K)
x, y = v.xy()

mde = hashlib.sha1(b"\x79" + int_to_bytes(osf, 2) + int_to_bytes(x, 64) + int_to_bytes(y, 64)).digest()
ep = int.from_bytes(mde[:4], byteorder="little")
ep &= 0x7fffffff

print(e, ep, e == ep)

Product Key (dashes optional): TCP8W-T8PQJ-WWRRH-QH76C-99FBW
TCP8W-T8PQJ-WWRRH-QH76C-99FBW
318 3e550ae1709773d8 6674d45a ce
f0ce4281d7695e3
110001100011111001010101000010101110000101110000100101110111001111011000110011001110100110101000101101000011001110
110001100011111001010101000010101110000101110000100101110111001111011000110011001110100110101000101101000011001110
1718932570 1718932570 True


In [28]:
0x8e/2

71